# Applicazione Python "Premi Nobel"

Attività scelta: 3. MongoDB Usecase e applicazione Python <br>
Progetto realizzato da Elena Curti (matricola: 146539) con Python 3.9.13.

Ho usato i seguenti file JSON pubblici: <br>
    <ul> 
      <li> nobelPrizes.json: https://masterdataapi.nobelprize.org/2.1/nobelPrizes?offset=0&limit=664 </li>
      <li> laureates.json: https://masterdataapi.nobelprize.org/2.1/laureates?offset=0&limit=981 </li>
    </ul>
Essi contengono informazioni sui premi Nobel ed i relativi vincitori.

## Documentazione 
Documentazione dei dataset: https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1

## File con i premi Nobel
Il file <i>JSON_Files/modificati/nobelPrizes2.json</i> contiene l'elenco dei premi Nobel, con la seguente struttura d'esempio:

```
{
    "awardYear": 2021,
    "category": "Economic Sciences",
    "categoryFullName": "The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel",
    "dateAwarded": "2021-10-11",
    "topMotivation": "for contributions ...",
    "prizeAmount": 10000000,
    "laureates": [
        {
            "id": "1007",
            "knownName": "David Card", // Solo per le persone
            "orgName": "Institute of International Law", // Solo per le organizzazioni
            "portion": "1/2",
            "sortOrder": "1",
            "motivation": "for his empirical contributions to labour economics"
        },
        { ... }
    ]
}
```

### Modifiche effettuate
Rispetto al file JSON originale (contenuto in <i>JSON_Files/originali/nobelPrizes.json</i>): 
<ul>
    <li> Ho rimosso i sottoalberi "/link" e "/meta" perchè contenenti informazioni non inerenti ai premi Nobel. <br></li>
    <li> 
        Ho rimosso la traduzione di alcuni campi. La struttura originale di tali campi era la seguente: 

```
"nome": {
    "en": "termine_inglese",
    "no": "termine_svedese",
    "se": "termine_norvegese"
}

```

<ul> 
<li style="list-style-type: none; ">
    Ho deciso di lasciare solo il termine inglese, trasformando quindi i campi in formato testuale. <br> 
    Ho eseguito questa modifica per i campi: "category", "categoryFullName", "laureates.knownName" e "laureates.motivation".
</li>
<li> Ho trasformato "awardYear" in un campo intero (originariamente testuale)</li>
<li> Ho rimosso i campi "laureates.fullName" e "laureates.nativeName" (il motivo è spiegato in seguito) </li>
</ul>

## File con i vincitori
Il file <i>JSON_Files/modificati/laureates2.json</i> contiene l'elenco dei vincitori dei premi Nobel, con la seguente struttura d'esempio:

```
// Per le persone
{
    "id": "102",
    "knownName": "Aage N. Bohr",
    "givenName": "Aage N.",
    "familyName": "Bohr",
    "fullName": "Aage Niels Bohr",
    "gender": "male",
    "birth": {
        "date": "1922-06-19",
        "place": {
            "cityNow": "Copenhagen",
            "countryNow": "Denmark"
        }
    },
    "death": {
        "date": "2009-09-08",
        "place": {
            "cityNow": "Copenhagen",
            "countryNow": "Denmark"
        }
    },
    "wikipedia": "https://en.wikipedia.org/wiki/Aage_Bohr"
} 

// Per le organizzazioni
{
    "id": "537",
    "orgName": "Amnesty International",
    "nativeName": "Amnesty International",
    "founded": {
        "date": "1961-00-00",
        "place": {
            "cityNow": "London",
            "countryNow": "United Kingdom"
        }
    },
    "wikipedia": "https://en.wikipedia.org/wiki/Amnesty_International"
}
```

### Modifiche effettuate
Rispetto al file JSON originale (contenuto in <i>JSON_Files/originali/laureates.json</i>): 
<ul>
    <li>  
        Ho rimosso i sottoalberi "/link" e "/meta" perchè contenenti informazioni non inerenti ai vincitori.
    </li>
    <li> 
        Come prima, ho rimosso la traduzione svedese e norvegese dei seguenti campi (lasciando solo quella inglese): "knownName", "givenName", "givenName", "fullName", "cityNow", "countryNow", "orgName"
    </li>
    <li>
        Ho modificato i campi "birth.place", "death.place" e "founded.place" rimuovendo i sotto-campi "city", "country", "continent" e "locationString". In questo modo ho semplificato la struttura del file, rimuovendo informazioni molto specifiche sui luoghi geografici e non inerenti al vincitore del premio.
    </li>
    <li> Ho rimosso il campo "nobelPrizes", contenente l'elenco dei premi Nobel vinti dalla persona/organizzazione, perchè già memorizzati nel file nobelPrizes2.json.
</ul>

## Struttura delle Collection
Nelle interrogazioni proposte nell'applicazione, ho dato la possibilità all'utente di scegliere se visionare i vincitori dei premi in modo sintetico (solo il nome) o dettagliato (tutti i campi). Quindi, ho deciso di non unire i file laureates2.json e nobelPrizes2.json, ma di tenerli distinti. Ho deciso, inoltre, di lasciare in nobelPrizes2.json solo i campi "laureates.knownName" e "laureates.orgName", rimuovendo quindi fullName e nativeName. <br>

## Applicazione
Eseguire il codice nelle celle, seguendo l'ordine proposto. <br>

### Operazioni iniziali e definizione di funzioni

Inserire nella seguente variabile il path della cartella con i file JSON:

In [2]:
import os
path_cartella_file_json =  "JSON_Files" + os.sep + "modificati" + os.sep
# DA TOGLIERE:
path_cartella_file_json = 'C:\\Users\\utente\\Desktop\\JSON_Nobel\\'

Il seguente codice crea ed inizializza un database chiamato <i>premi_nobel</i>, formato dalle due collections <i>laureates2</i> e <i>nobelPrizes2</i> (contenenti i dati memorizzati nei relativi file JSON). <br>
<b>NB: Eseguedo questo codice, gli eventuali dati memorizzati nelle collections e non presenti nei file JSON saranno eliminati. </b> Eseguire quindi il codice nel primo avvio dell'applicazione o quando si vuole ripristinare il database.

In [3]:
import json, sys
from pymongo import MongoClient

class MiaStopExecution(Exception):
    """Il raise di questa classe provoca l'interruzione dell'esecizione della cella, senza interrompere il kernel"""
    def _render_traceback_(self):
        pass

def carica_dati(nomeFile, collection):
    ''' Funzione che carica il file json nella collection. '''
    collection.drop()
    try:
        with open(path_cartella_file_json+nomeFile, encoding='utf8') as file:
            file_data = json.load(file)
    except FileNotFoundError:
        print("Path inserito non valido! Assicurarsi di aver messo / o \\\\ alla fine del path.", file=sys.stderr)
        raise MiaStopExecution
    collection.insert_many(file_data) 

# Costanti
FILE_LAUREATES = "laureates2.json"
FILE_NOBEL_PRICES = "nobelPrizes2.json"
LOOKUP_JOIN_COLLECTIONS = { 
            '$lookup': { 
                "localField": "laureates.id",
                "from": "laureates2",
                "foreignField": "id", 
                "as": "laureates" 
            } 
        }

# Inizializzazione del database
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient["premi_nobel"]
collection_laureates2 = db["laureates2"]
collection_nobelPrices2 = db["nobelPrizes2"]

carica_dati(FILE_LAUREATES, collection_laureates2) 
carica_dati(FILE_NOBEL_PRICES, collection_nobelPrices2)
print("Database inizializzato correttamente")

Database inizializzato correttamente


In [4]:
def get_info_event(dizionario):
    """Funzione che ritorna l'evento in input serializzato in una stringa"""
    ret_str = ""
    if "date" in dizionario:
        ret_str += dizionario["date"]
    if "place" in dizionario:
        ret_str += " a "+ dizionario["place"].get("cityNow", "") + ", " +dizionario["place"].get("countryNow", "")
    return ret_str


def print_vincitore(l, stampa_dettagli):
    """Funzione che stampa i dati del vincitore in input"""
    if "knownName" in l:
        print("Persona: " + str(l["knownName"]))
    if "orgName" in l:
        print("Organizzazione: " + str(l["orgName"]))

    if not stampa_dettagli:
        return
    # Dettagli persone
    if "givenName" in l or "familyName" in l or "fullName" in l:
        print("\tAltri nomi: " + str(l.get("fullName", '')) + " - " + str(l.get("givenName", '')) +" - " + str(l.get("familyName", '')))
    if "gender" in l:
        print("\tSesso: " + str(l["gender"])[0])
    if "birth" in l:
        print("\tNato/a il " + get_info_event(l["birth"]))
    if "death" in l:
        print("\tMorto/a il " + get_info_event(l["death"]))
    
    # Dettagli organizzazioni
    if "nativeName" in l:
        print("\tAltri nomi: " + str(l["nativeName"]))
    if "founded" in l:
        print("\tFondato/a il " + get_info_event(l["founded"]))
    
    # Dettagli comuni
    if "wikipedia" in l:
        print("\tWikipedia: " + str(l["wikipedia"]))

def print_nobels(nobel_list, stampa_dettagli):
    """Funzione che stampa il nobel dato in input (in modo dettagliato se stampa_dettagli e' true)."""
    
    def print_nobel(nobel):
        print("Anno: " + str(nobel.get('awardYear', "-")))
        print("Categoria: " + str(nobel.get('category', "-")) + " - anche detta " + str(nobel.get('categoryFullName', "")))
        print("Data: " + str(nobel.get('dateAwarded', "-")))
        print("Motivazione: " + str(nobel.get('topMotivation', "-")))
        if "prizeAmount" in nobel:
            prizeAmount = nobel['prizeAmount']
            print("Importo del premio (in corone svedesi): " + get_importo(prizeAmount))
        laureates = nobel.get('laureates', [])
        for l in laureates:
            print_vincitore(l, stampa_dettagli)
            if 'portion' in l:
                print("\tPorzione del Nobel vinta: " +  str(l['portion']))
            if 'motivation' in l:
                print("\tMotivazione: " +  str(l['motivation']))

    
    for nobel in nobel_list:
        print("*************")
        print_nobel(nobel)

def chiedi_stampa_dettagli():
    return input("Stampare anche i dettagli dei vincitori (y/n)?").lower()=='y'

def get_importo(importo):
    return str(f"{importo:,.2f}")

### Interrogazioni

#### 1. Cercare tutti i vincitori dei premi Nobel di una categoria

In [4]:
print("Cercare tutti i vincitori dei premi Nobel di una categoria")

# Stampa del menu e richiesta degli input
print("Quale categoria? ")
lista_categorie = collection_nobelPrices2.distinct("category") 
i = 0
for cat in lista_categorie:
    print(str(i)+ " -- " + cat)
    i+=1
num_categoria = int(input(">> "))
stampa_dettagli = input("Stampare anche i dettagli dei vincitori (y/n)?").lower()=='y'

# Controllo dei dati in input
try:
    print("Scelta: " + lista_categorie[num_categoria] + " "+("con" if stampa_dettagli else "senza" ) + " dettagli")
except IndexError:
    print("Numero inserito non valido!")
    raise MiaStopExecution

# Interrogazione e stampa dei risultati
if stampa_dettagli:
    lista_nobel = collection_nobelPrices2.aggregate([
        {'$match': {"category":lista_categorie[num_categoria]}}, 
        LOOKUP_JOIN_COLLECTIONS
    ])
else:
    lista_nobel = collection_nobelPrices2.find({"category":lista_categorie[num_categoria]}) 

print_nobels(lista_nobel, stampa_dettagli)

Cercare tutti i vincitori dei premi Nobel di una categoria
Quale categoria? 
0 -- Chemistry
1 -- Economic Sciences
2 -- Literature
3 -- Peace
4 -- Physics
5 -- Physiology or Medicine
Scelta: Chemistry senza dettagli
*************
Anno: 1901
Categoria: Chemistry - anche detta The Nobel Prize in Chemistry
Data: -
Motivazione: -
Importo del premio (in corone svedesi): 150,782.00
Persona: Jacobus H. van 't Hoff
	Porzione del Nobel vinta: 1
	Motivazione: in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions
*************
Anno: 1902
Categoria: Chemistry - anche detta The Nobel Prize in Chemistry
Data: -
Motivazione: -
Importo del premio (in corone svedesi): 141,847.00
Persona: Emil Fischer
	Porzione del Nobel vinta: 1
	Motivazione: in recognition of the extraordinary services he has rendered by his work on sugar and purine syntheses
*************
Anno: 1903
Categoria: Chemistry - anche detta The Nobel 

#### 2. Stampare i primi Nobel in ordine (crescente o decrescente) di importo o data

In [5]:
print("Stampare i primi N Nobel degli in ordine (crescente o decrescente) di importo o di data ")

# Richiesta degli input
n_str = input("Quanti nobel stampare (premere invio per stamparli tutti)?")
try:
    n = int(n_str if n_str != "" else -1)
    if n!=-1 and n<1:
        raise ValueError
except ValueError:
    print("Errore! Numero inserito non valido")
    raise MiaStopExecution

importo_o_data=""
while(not (importo_o_data=="1" or importo_o_data=="2")):
    importo_o_data = input("Si desiderano vedere i nobel in ordine di:\n1.Importo del premio\n2.Data\n>> ")
importo_o_data = "prizeAmount" if importo_o_data == "1" else "dateAwarded"

ordine=""
while(not (ordine=="1" or ordine=="2")):
    ordine = input("Si desiderano vedere i nobel in ordine:\n1.Crescente\n2.Decrescente\n>> ")
ordine = 1 if ordine == "1" else -1

stampa_dettagli = chiedi_stampa_dettagli()

print("Hai scelto: " + 
    ("tutti i nobel" if n==-1 else "primi " + n_str + " nobel" ) 
    + " in ordine " + ("crescente" if ordine == 1 else "decrescente") + " di "
    + ("data" if importo_o_data == "dateAwarded" else "importo") + " " 
    + ("con" if stampa_dettagli else "senza" ) + " dettagli"
)

# Interrogazione e stampa dei risultati
if stampa_dettagli:
    lista_filtri = [
        { '$match': {importo_o_data:{'$exists':'true'}}},    
        { '$sort':  {importo_o_data: ordine } },
        LOOKUP_JOIN_COLLECTIONS
    ]
    if n!=-1:
        lista_filtri += [{'$limit':n}]
    lista_nobel = collection_nobelPrices2.aggregate(lista_filtri)
else:
    lista_nobel = collection_nobelPrices2.find({importo_o_data:{'$exists':'true'}}).sort(importo_o_data, ordine)
    if n != -1:
        lista_nobel = lista_nobel.limit(n)

print_nobels(lista_nobel, stampa_dettagli)

Stampare i primi N Nobel degli in ordine (crescente o decrescente) di importo o di data 
Hai scelto: primi 2 nobel in ordine decrescente di importo con dettagli
*************
Anno: 2001
Categoria: Economic Sciences - anche detta The Sveriges Riksbank Prize in Economic Sciences in Memory of Alfred Nobel
Data: 2001-10-10
Motivazione: -
Importo del premio (in corone svedesi): 10,000,000.00
Persona: A. Michael Spence
	Altri nomi: A. Michael Spence - A. Michael - Spence
	Sesso: m
	Nato/a il 1943-00-00 a Montclair, NJ, USA
	Wikipedia: https://en.wikipedia.org/wiki/Michael_Spence
Persona: George A. Akerlof
	Altri nomi: George A. Akerlof - George A. - Akerlof
	Sesso: m
	Nato/a il 1940-06-17 a New Haven, CT, USA
	Wikipedia: https://en.wikipedia.org/wiki/George_Akerlof
Persona: Joseph E. Stiglitz
	Altri nomi: Joseph E. Stiglitz - Joseph E. - Stiglitz
	Sesso: m
	Nato/a il 1943-02-09 a Gary, IN, USA
	Wikipedia: https://en.wikipedia.org/wiki/Joseph_Stiglitz
*************
Anno: 2001
Categoria: Chemi

#### 3. Interrogazioni con max, min e avg

In [6]:
print("Interrogazioni con max, min e avg")
from math import trunc

# Interrogazione e stampa dei risultati
risultati = list(collection_nobelPrices2.aggregate([{ "$group": { 
        "_id": 'null',
        "max_data": { "$max": "$dateAwarded" } , 
        "min_data": { "$min": "$dateAwarded" } ,
        "max_prize": { "$max": "$prizeAmount" } , 
        "min_prize": { "$min": "$prizeAmount" } ,
        "avg_prize": { "$avg": "$prizeAmount"}
    }}]))[0]
print("I nobel presenti nel database:")
print("\t- sono stati vinti tra il " + risultati['min_data'] + " e il " + risultati["max_data"])
print("\t- hanno un importo compreso tra " + get_importo(risultati['min_prize']) + " e " + get_importo(risultati['max_prize']) + " (corone svedesi), per un importo medio di: " 
    + get_importo(risultati['avg_prize']))

Interrogazioni con max, min e avg
I nobel presenti nel database:
	- sono stati vinti tra il 1901-12-10 e il 2022-10-07
	- hanno un importo compreso tra 114,935.00 e 10,000,000.00 (corone svedesi), per un importo medio di: 2,866,560.87


#### 4. Quanti Nobel sono stati vinti per ogni categoria?

In [7]:
print("Quanti Nobel sono stati vinti per ogni categoria?\n")
print("Categoria\t\t\tNumero di Nobel vinti")
print("--------------------------------------------------------")

category_count = list(collection_nobelPrices2.aggregate([   
    { '$unwind': "$category" },
    { '$group': { '_id': "$category" , 'count': { '$sum': 1 } }}
]))

for elem in category_count:
    categoria = elem["_id"] 
    count = str(elem["count"])
    sep = "\t\t" if len(categoria)>16 else ("\t\t\t\t" if len(categoria)<9 else "\t\t\t")
    print(categoria +sep+ count)

Quanti Nobel sono stati vinti per ogni categoria?

Categoria			Numero di Nobel vinti
--------------------------------------------------------
Physiology or Medicine		122
Economic Sciences		54
Peace				122
Chemistry			122
Literature			122
Physics				122


#### 5. Persone/organizzazioni che hanno vinto piu di un Nobel

In [10]:
print("Persone/organizzazioni che hanno vinto piu di un Nobel")

# Richiesta degli input
print("Scegliere un'opzione:\n1. Considerare solo i vincitori che hanno vinto come singoli\n2. Considerare anche coloro che hanno vinto insieme ad altri vincitori")
opzione = ""
while (not(opzione=="1" or opzione=="2")):
    opzione = input(">> ")

stampa_dettagli = chiedi_stampa_dettagli()

# Interrogazione e stampa dei risultati
filtri_ricerca = [
    { '$group': { '_id': "$laureates.id" , 'count': { '$sum': 1 } }},
    { '$match' : {'count': {'$gt':1}}}, 
    { '$sort' : {'count':-1}},
    { '$lookup': { 
            "localField": "_id",
            "from": "laureates2",
            "foreignField": "id", 
            "as": "vincitore" 
        } 
    }
]
if opzione == "2":
    filtri_ricerca = [{ '$unwind': "$laureates" }] + filtri_ricerca

lista_vincitori = list(collection_nobelPrices2.aggregate(filtri_ricerca))

for elem in lista_vincitori:
    if len(elem["vincitore"]) == 0:
        continue
    print("--------------")
    print("Nobel vinti: " + str(elem["count"]))
    print_vincitore(elem["vincitore"][0], stampa_dettagli)


Persone/organizzazioni che hanno vinto piu di un Nobel
Scegliere un'opzione:
1. Considerare solo i vincitori che hanno vinto come singoli
2. Considerare anche coloro che hanno vinto insieme ad altri vincitori
--------------
Nobel vinti: 3
Organizzazione: International Committee of the Red Cross
--------------
Nobel vinti: 2
Persona: Marie Curie
--------------
Nobel vinti: 2
Organizzazione: Office of the United Nations High Commissioner for Refugees
--------------
Nobel vinti: 2
Persona: Frederick Sanger
--------------
Nobel vinti: 2
Persona: John Bardeen
--------------
Nobel vinti: 2
Persona: K. Barry Sharpless
--------------
Nobel vinti: 2
Persona: Linus Pauling


#### 6. Sesta interrogazione
- Un vincitore italiano random 
- Trasformare in euro i premi
- usando cond
- Trovare gli anni in cui non e' stato dato nessun nobel:

db.getCollection('nobelPrizes2').aggregate([
        { '$match': {"laureates.id" : {$eq: null}} }, 
        { '$group': {
            "_id":'$awardYear', 
            'count':{'$sum':1} , 
            'categorie': {
                '$accumulator': {
                    accumulateArgs: ["$category"],
                    init: function() { return [] },
                    accumulate: function(ids, id) { return ids.concat(id) },
                    merge: function(ids1, ids2) { return ids1.concat(ids2) },
                    finalize: function(ids) { return ids.join(",") },
                    lang: "js"
                 }
                 }
         }}, 
         { '$sort': {'count':-1}},
        
        //{ '$group': {"_id":null}
        //{ $project: { newfield: { $concat: [ "$field1", " - ", "$field2" ] } } }

]).toArray()
<mark> DA FARECon comandi non visti a lezione

In [18]:
import pprint 
# pprint.pprint(list(collection_laureates2.find(
#     { '$and': [
#         {'$or':[{"birth.place.countryNow":"Italy"}, {"founded.place.countryNow":"Italy"}]} ,
#         {'$or':[{"death.place.countryNow":"Italy"}, { 'death': {'$exists':'false'}}]}
#     ]}

#     ) ))

pprint.pprint(list(collection_nobelPrices2.find(
    {'laureates.id': {'$exists':'false'}}

    )))

[{'_id': ObjectId('636b794d299e321f1396d72d'),
  'awardYear': 1901,
  'category': 'Chemistry',
  'categoryFullName': 'The Nobel Prize in Chemistry',
  'laureates': [{'id': '160',
                 'knownName': "Jacobus H. van 't Hoff",
                 'motivation': 'in recognition of the extraordinary services '
                               'he has rendered by the discovery of the laws '
                               'of chemical dynamics and osmotic pressure in '
                               'solutions',
                 'portion': '1',
                 'sortOrder': '1'}],
  'prizeAmount': 150782},
 {'_id': ObjectId('636b794d299e321f1396d72e'),
  'awardYear': 1901,
  'category': 'Literature',
  'categoryFullName': 'The Nobel Prize in Literature',
  'laureates': [{'id': '569',
                 'knownName': 'Sully Prudhomme',
                 'motivation': 'in special recognition of his poetic '
                               'composition, which gives evidence of lofty '
          

### Scritture e modifiche

In [11]:
def chiedi_id_nome_vincitore():
    id = ""
    while id == "":
        name = input("Inserire il nome della persona/organizzazione: ")
        lista_ris = list(collection_laureates2.find({'$or': [{"knownName" : name}, {"orgName" : name}]} ))
        
        if len(lista_ris) ==0:
            print("Nessun risultato trovato. Reinserire il nome. ")
            continue

        print("Risultati trovati:")
        id_list = []
        for vincitore in lista_ris:
            print_vincitore(vincitore, True)
            if "id" in vincitore:
                print("\tId:" + vincitore["id"])                    
                id_list += [vincitore["id"]]
            print()
        id = input("Inserire il numero di id per confermare (premere invio per reinserire il nome): ")
        
        if id not in id_list:
            print("Id inserito non presente nella lista! Reinserire il nome. ")
            id = ""
        vincitore = list(collection_laureates2.find({'id':id}))[0]
        if "orgName" in vincitore:
            id_known_or_org_name = {"id": id, "orgName": vincitore["orgName"]}
        elif "knownName" in vincitore:
            id_known_or_org_name = {"id": id, "knownName": vincitore["knownName"]}
        return id_known_or_org_name
        

#### 1. Inserire un premio nobel ed eventualmente il vincitore

In [12]:
print("Inserire dati di un nuovo premio Nobel")
print("I dati non conosciuti possono essere omessi premendo invio")
from datetime import datetime

def check_data(data_str):
    """Questa funzione controlla che la data in input abbia un formato corretto"""
    if data_str != "":
        datetime.strptime(data_str, "%Y-%m-%d")

def inserisci_nuovo_vincitore():
    """Questa funzione inserisce nella collection un nuovo vincitore"""
    
    # Richiesta e controlli degli input relativi ai vincitori
    is_persona = input("Il nobel e' stato vinto da una persona (y/n)?") == 'y'
    if is_persona:
        knownName = input("Nome (principale): ")
        givenName = input("Altro nome: ")
        familyName = input("Nome della famiglia: ")
        fullName = input("Nome completo: ")
        gender = input("Sesso (m/f): ") 
        gender = "female" if gender == 'f' else "male"
        birthDate = input("Data di nascita ("+FORMATO_DATA+"): ")
        check_data(birthDate)
        birthPlace_cityNow = input("Citta di nascita: ")
        birthPlace_countryNow = input("Paese: ")

        deathDate = input("Data di morte ("+FORMATO_DATA+"): ")
        check_data(deathDate)
        deathPlace_cityNow = input("Citta di morte: ")
        deathPlace_countryNow = input("Paese: ")
    else:
        orgName = input("Nome (principale): ")
        nativeName = input("Altro nome: ")
        foundedDate = input("Data di fondazione ("+FORMATO_DATA+"): ")
        check_data(foundedDate)
        foundedPlace_cityNow = input("Citta di fondazione: ")
        foundedPlace_countryNow = input("Paese: ")

    wikipedia = input("Link alla pagina wikipedia: ")
    id = str(collection_laureates2.count()+1025)
    
    # Inserimento del documento e return 
    if is_persona:
        collection_laureates2.insert_one({
            "id" : id, 
            "knownName": knownName,
            "givenName": givenName,
            "familyName": familyName,
            "fullName": fullName,
            "gender": gender,
            "birth": {
                "date": birthDate,
                "place": {
                    "cityNow": birthPlace_cityNow,
                    "countryNow": birthPlace_countryNow
                }
            },
            "death": {
                "date": deathDate,
                "place": {
                    "cityNow": deathPlace_cityNow,
                    "countryNow": deathPlace_countryNow
                }
            },
            "wikipedia": wikipedia
        })
        return {"id" : id, "knownName": knownName }
    else: 
        collection_laureates2.insert_one({
            "id": id,
            "orgName": orgName,
            "nativeName": nativeName,
            "founded": {
                "date": foundedDate,
                "place": {
                    "cityNow": foundedPlace_cityNow,
                    "countryNow": foundedPlace_countryNow
                }
            },
            "wikipedia": wikipedia
        })
        return  {"id" : id, "orgName": orgName } 
    
        
# Richiesta dei valori relativi al premio Nobel
FORMATO_DATA = "aaaa-mm-gg"

awardYear = input("Anno: ")
awardYear = int(awardYear) if awardYear != "" else ""

category = input("Categoria: ")

categoryFullName = input("Nome completo della categoria: ")

dateAwarded = input("Data ("+FORMATO_DATA+"): ")
check_data(dateAwarded)

topMotivation = input("Motivazione: ")

prizeAmount = input("Importo (in corone svedesi): ")
prizeAmount = int(prizeAmount) if prizeAmount != "" else ""


n_laureates = int(input("Numero di vincitori: "))
lista_laureates = []

for i in range(n_laureates):
    # Richiesta dei dati dei vincitori
    print(f"---Dati del vincitore {i+1}---")
    giaPresente = input("Il vincitore e' gia presente (y/n)? ") == 'y'
    if giaPresente:
        id_known_or_org_name = chiedi_id_nome_vincitore()
    else:
        id_known_or_org_name = inserisci_nuovo_vincitore()

    portion = input("Porzione del Nobel vinta (frazione): ")
    motivation = input("Motivazione: ")

    lista_laureates += [id_known_or_org_name|{"portion":portion, "motivation":motivation}]

# Inserimento del documento
collection_nobelPrices2.insert_one({
    "awardYear": awardYear,
    "category": category,
    "categoryFullName": categoryFullName,
    "dateAwarded": dateAwarded,
    "topMotivation": topMotivation,
    "prizeAmount": prizeAmount,
    "laureates": lista_laureates
})    
print("Tutti i dati sono stati memorizzati correttamente!")

Inserire dati di un nuovo premio Nobel
I dati non conosciuti possono essere omessi premendo invio
---Dati del vincitore 1---


C:\Users\utente\AppData\Local\Temp\ipykernel_16804\594976240.py:40: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  id = str(collection_laureates2.count()+1025)


Tutti i dati sono stati memorizzati correttamente!


#### 2. Modifica dei dati di un vincitore

In [13]:
print("Modifica dei dati di un vincitore")

# Richiesta degli input
id_known_or_org_name = chiedi_id_nome_vincitore()
is_persona = "knownName" in id_known_or_org_name
if is_persona:
    lista_field = [ ("knownName", "Persona"), 
            ("fullName", "Altro nome 1:"), 
            ("givenName", "Altro nome 2"), 
            ("familyName", "Altro nome 3"), 
            ("gender", "Sesso (m/f)"), 
            ("birth.date", "Data di nascita"),
            ("birth.place.cityNow", "Citta di nascita"),
            ("birth.place.countryNow", "Paese di nascita"),
            ("death.date", "Data di morte"),
            ("death.place.cityNow", "Citta di morte"),
            ("death.place.countryNow", "Paese di morte"),
            ("wikipedia", "Link di wikipedia")
        ]
else:
    lista_field = [
        ("orgName", "Organizzazione"), 
        ("nativeName", "Altro Nome"), 
        ("founded.date", "Data di fondazione"),
        ("founded.place.cityNow", "Citta di fondazione"),
        ("founded.place.countryNow", "Paese di fondazione"),
        ("wikipedia", "Link di wikipedia")
    ]

print("Che valore si desidera aggiungere o modificare?")
for i in range(len(lista_field)):
    print(f"{i}. " + lista_field[i][1])
opzione = int(input(">> "))
if opzione < 0 or opzione >= len(lista_field):
    print("Valore non valido!")
    raise MiaStopExecution

nuovo_val = input("Inserire il nuovo valore: ")

if lista_field[opzione][0].__contains__("date"):
    check_data(nuovo_val)
elif lista_field[opzione][0] == "gender":
    gender = "female" if nuovo_val == 'f' else "male"

# Modifico i dati
collection_laureates2.update_one({'id':id_known_or_org_name["id"]},{'$set':{lista_field[opzione][0]:nuovo_val}})
print("Documento memorizzato correttamente: ")
print_vincitore(list(collection_laureates2.find({'id':id_known_or_org_name["id"]}))[0], True)

Modifica dei dati di un vincitore
Risultati trovati:
Organizzazione: Amnesty International
	Altri nomi: Amnesty International
	Fondato/a il 1961-00-00 a London, United Kingdom
	Wikipedia: https://en.wikipedia.org/wiki/Amnesty_International
	Id:537

Che valore si desidera aggiungere o modificare?
0. Organizzazione
1. Altro Nome
2. Data di fondazione
3. Citta di fondazione
4. Paese di fondazione
5. Link di wikipedia
Documento memorizzato correttamente: 
Organizzazione: Amnesty International
	Altri nomi: abcs
	Fondato/a il 1961-00-00 a London, United Kingdom
	Wikipedia: https://en.wikipedia.org/wiki/Amnesty_International
